*This Notebook has been created by PALISSON Antoine.*<br>


In [1]:
# Base packages import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Datasets

pmdarima datasets [Documentation](https://alkaline-ml.com/pmdarima/modules/classes.html#pmdarima-datasets-toy-timeseries-datasets)<br>

In [2]:
# You need to re-install statsmodel before installing pmdarim on google colab
# On your local machine, be careful on the versions you are using
!pip install -q statsmodels
!pip install -q pmdarima
from pmdarima.datasets import load_airpassengers, load_msft, load_ausbeer, load_austres


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
airp_df = load_airpassengers(as_series=True)
airp_df.index = pd.to_datetime(pd.date_range(start='01/01/1949', periods=len(airp_df), freq='M'))
airp_df.name = 'Air Passenger'

msft_df = load_msft()
msft_df = pd.Series(msft_df['Close'].to_numpy(), index=msft_df['Date'])
msft_df.index = pd.to_datetime(msft_df.index)

# Simple RNN

Simple RNN layers can be used whenever a Time Series does not require the model to remember information from long ago.

In TensorFlow Keras, the SimpleRNN layer has several important parameters that you can use to configure the behavior of the layer. Some of the most important parameters include:

* **units**: This parameter specifies the number of hidden units in the SimpleRNN layer. The number of hidden units determines the size of the internal representation of the input data, and can affect the model's ability to capture complex patterns in the data. High values may harm the performances for simple datasets whereas low values may harm the performances for complex datasets.

* **activation**: This parameter specifies the activation function to use for the output of the SimpleRNN layer. Common activation functions include "relu", "tanh", and "sigmoid". In general, it's a good idea to try different activation functions and compare the performance of the model to see which one works best.

* **return_sequences**: This parameter determines whether the output of the SimpleRNN layer should be returned for every timestep or just the last timestep. If set to True, the output will be returned for every timestep, and can be used as input to another RNN layer. If set to False, the output will only be returned for the last timestep, and is typically used as input to a dense layer.

* **return_state**: This parameter determines whether the hidden state of the SimpleRNN layer should be returned. If set to True, the hidden state will be returned in addition to the output, and can be used to initialize the state of another RNN layer. If set to False, the hidden state will not be returned.

* **input_shape**: This parameter specifies the shape of the input data, in the form of (batch_size, timesteps, input_dim). It is only required when creating the first layer of the model.

* **kernel_initializer** and recurrent_initializer : These parameters are used to initialize the weight matrices of the SimpleRNN layer. You can use different initializers to change the behavior of the model and to improve the training performance.

* **kernel_regularizer** and **recurrent_regularizer**: These parameters are used to apply weight regularization to the SimpleRNN layer. Regularization can help prevent overfitting by adding a penalty term to the loss function based on the magnitude of the weights.

* **dropout**: This parameter applies dropout regularization to the output of the SimpleRNN layer. Dropout randomly sets a fraction of the input units to 0 at each update during training time, which helps prevent overfitting by reducing the number of parameters that can be updated.

In general, a deeper model (more layers) can potentially capture more complex patterns in the data, but it can also increase the risk of overfitting. On the other hand, a shallower model (fewer layers) may not be able to capture as many complex patterns, but it may be less prone to overfitting.

A good way to decide the number of layers is to start with a small number of layers and gradually increase the number while monitoring the performance of the model on a validation set. You can also try to use early stopping, which is a technique that can help you decide when to stop training by monitoring the performance on a validation set.

`SimpleRNN` [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN)

## One step at a time

In this exercise, we will predict the next value based and the previous ones.

**1. Split the airp_df time series into a train and a test set without shuffling.**

*Tips: We will create the validation set later on.*

In [15]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(airp_df,test_size = 0.25, shuffle=False)


In [10]:
test

1958-01-31    340.0
1958-02-28    318.0
1958-03-31    362.0
1958-04-30    348.0
1958-05-31    363.0
1958-06-30    435.0
1958-07-31    491.0
1958-08-31    505.0
1958-09-30    404.0
1958-10-31    359.0
1958-11-30    310.0
1958-12-31    337.0
1959-01-31    360.0
1959-02-28    342.0
1959-03-31    406.0
1959-04-30    396.0
1959-05-31    420.0
1959-06-30    472.0
1959-07-31    548.0
1959-08-31    559.0
1959-09-30    463.0
1959-10-31    407.0
1959-11-30    362.0
1959-12-31    405.0
1960-01-31    417.0
1960-02-29    391.0
1960-03-31    419.0
1960-04-30    461.0
1960-05-31    472.0
1960-06-30    535.0
1960-07-31    622.0
1960-08-31    606.0
1960-09-30    508.0
1960-10-31    461.0
1960-11-30    390.0
1960-12-31    432.0
Freq: M, Name: Air Passenger, dtype: float64

The input shape of a SimpleRNN model in TensorFlow Keras is a tuple of three integers that specify the dimensions of the input data, in the form of (batch_size, timesteps, input_dim).

* **batch_size**: This is the number of samples per gradient update. It corresponds to the number of samples in a batch. For example, if you have 1000 samples and a batch size of 32, it will take 32 gradient updates to process all of the samples.

* **timesteps**: This is the number of time steps in the input data. In the case of time series data, the timesteps represent the number of observations of the time series at a specific time interval. The number of time steps is also related to the length of the sequence you want to process.

* **input_dim**: This is the number of features in the input data. It corresponds to the number of input dimensions in the input data. For example, if you are processing text data, it corresponds to the number of unique words in the vocabulary.

For example, if you are working with a window size of 12 for a Time Series without any exogenous variable, the shape of the X dataset should be set to (Batch Size, 12, 1). Additionally, if you want to predict the next value, the Y dataset should be set to (Batch Size, 1).

`[____XXXXXY______]`<br>
`[_____XXXXXY_____]`<br>
`[______XXXXXY____]`<br>
`[_______XXXXXY___]`<br>

**2. Create the feature (X) set by using a sliding windows of size 12, with a step of 1 for both train and test sets. Create the label (Y) set by using the next value i.e. the one after the window.**

In [19]:
def to_sequences(data, win_length, win_offset,target_length):
    x= []
    y= []
    max_length = len(data - win_length - target_length + 1 )

    for i in range(0, max_length, win_offset):
        x.append(data[i : i + win_length])
        y.append(data[i + win_length:i + win_length + target_length])
    return np.array(x)[..., np.newaxis], np.array(y)
    
seq = to_sequences(airp_df, 12, 1, 12)

seq

/var/folders/60/bfzc0kqx1l54g6mx713k6cx80000gn/T/ipykernel_13304/670049709.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(x)[..., np.newaxis], np.array(y)


(array([[1949-01-31    112.0
         1949-02-28    118.0
         1949-03-31    132.0
         1949-04-30    129.0
         1949-05-31    121.0
         1949-06-30    135.0
         1949-07-31    148.0
         1949-08-31    148.0
         1949-09-30    136.0
         1949-10-31    119.0
         1949-11-30    104.0
         1949-12-31    118.0
         Freq: M, Name: Air Passenger, dtype: float64],
        [1949-02-28    118.0
         1949-03-31    132.0
         1949-04-30    129.0
         1949-05-31    121.0
         1949-06-30    135.0
         1949-07-31    148.0
         1949-08-31    148.0
         1949-09-30    136.0
         1949-10-31    119.0
         1949-11-30    104.0
         1949-12-31    118.0
         1950-01-31    115.0
         Freq: M, Name: Air Passenger, dtype: float64],
        [1949-03-31    132.0
         1949-04-30    129.0
         1949-05-31    121.0
         1949-06-30    135.0
         1949-07-31    148.0
         1949-08-31    148.0
         1949-09-3

**3. Standardize the values of the Time Series.**

*Tips: Don't forget to only compute the calculations on the Train set but apply the standardization to every set.*

There are many ways to create a simple RNN Deep Learning model architecture using **`Tensorflow Keras`**. Here is one of the simplest method:



1.   Start by creating the model with the `tf.keras.Sequential()` class
2.   Add the `Input()` layer to the model with the `model.add()` method. The input layer should contain the shape of the training data (window_length, number of features)
3.   Add one `SimpleRNN()`. Don't forget to specify its hyperparameters.
4.   Add more layers. If you are stacking SimpleRNN layers, don't forget to turn the `return_sequences` parameter to True unless it is the last SimpleRNN before the output layer.
5.   Add the output layer. For a Time Series, it is generally a `Dense()` layers that does not have any activation. The number of units depends on the number of outputs needed.

Sequential [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)<br>
Dense [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)


**4.a. Define a sequential deep learning model architecture that contains one input layer, some SimpleRNN layers and one output layer.<br> 4.b. Show the architecture and the parameters of the model using the `model.summary()` method.**

In TensorFlow, the `compile()` method is used to configure the learning process before training a model. It is typically called after the model architecture has been defined and the model has been compiled with the optimizer, loss function, and metrics to be used during training. The `compile()` method takes several arguments such as optimizer, loss and metrics. These are used to configure the model's training process.

Adam Optimizer [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)<br>
MSE loss [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError)

**5. Compile the model by adding an optimizer, a loss and, optionally, a metric using the `model.compile()` method. <br>Set the learning_rate of the optimizer to 0.001**

*Tips: You can use the Adam optimizer and the mean squared error loss.*

The `fit()` function is used to train a model. It is typically called after the model architecture has been defined and the model has been compiled with the optimizer, loss function, and metrics to be used during training. 

It returns a history object that contains the loss and metric values for each epoch of training.

The fit() method in TensorFlow has several important parameters, including:

* **x**: This is the input data for the model, typically a numpy array or a dataset.
* **y**: This is the target data for the model, also typically a numpy array or a dataset. It contains the labels or values that the model should predict.
* **batch_size**: This is the number of samples to use in each iteration of training. The model will update its weights after every batch.
* **epochs**: This is the number of times the model will iterate over the entire dataset during training.
* **validation_data** : This is used to monitor the training process, it can be a tuple of x and y validation data.
* **validation_split** : This is used to monitor the training process, it is the ratio of the data that will be used as a validation set.
* **verbose**: This is used to control how much output is printed during training. A value of 0 means no output is printed, while a value of 1 means progress bars and updates are printed.

Other paramters exist. You can check the `fit()` [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit)

**6. Train the model by using the `model.fit()` method.<br>Specify a value for the number of `epochs` and the `batch_size`, and set the validation_split hyperparameter to 0.1**

*Tips: You can use an early stopping callback to avoir overfitting.*

In TensorFlow, the `predict()` method is used to generate predictions for new data using a trained model. It is typically called after the model has been trained using the `fit()` method. 

The `predict()` method takes one or more input data and returns the model's predictions for each input. The input data should be in the same format as the input data used during training.

predict() [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict) 

**7.a. Predict the test set using the `model.predict()`.<br>7.b. Plot the predictions on a graph along with the true data.<br>7.c. Compute the RMSE.**

## Multiple steps at a time

In this exercise, we will predict the next n-th values based and the previous ones.

Evertyhing is similar to the previous exercise but the the window creation is changed as follows:

`[____XXXXXYYY____]`<br>
`[_____XXXXXYYY___]`<br>
`[______XXXXXYYY__]`<br>
`[_______XXXXXYYY_]`<br>

Additionally, the output layer should now contain n-th units: one for each value we want to forecast.

**Do the same as the previous exercise.**

# Bidirectionnal RNN

In this exercise, we will use the bidirectional RNN.

The `Bidirectional()` class is a wrapper layer that applies a recurrent layer to the input in both directions, processing the input forwards and backwards to get a full representation of the input sequence. The most important parameters of this class are:

* **layer**: This is the recurrent layer instance to be wrapped. The layer should have a stateful=False.
* **merge_mode**: This parameter controls how the forward and backward outputs are combined. There are four options: 'concat', 'sum', 'ave', 'mul'
* **weights**: List of Numpy arrays to set as initial weights. The list should have 2 elements, of shape (input_dim, output_dim) and (output_dim,) for weights and biases respectively.
* **backward_layer** : Instance of the recurrent layer to be used for the backward pass. If not provided, it will be created by cloning the layer.

You can directly pass this layer around the `SimpleRNN` layer during the model architecture creation.

`Bidirectional` [Documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional)

## One step at a time


**Follow the same guidelines as the first exercise.**

## Multiple steps at a time

**Follow the same guidelines as the second exercise with a Bidirectional method.**

# LSTM

In this exercise, we'll use the `LSTM` class on the msft_df Time Series.

The most important parameters of this class are very similar to the SimpleRNN class, there are:

* **units**: This is the number of hidden units in the LSTM layer. It controls the dimensionality of the output space.
* **activation**: This is the activation function to use for the LSTM layer. Common options include 'tanh' and 'relu'.
* **recurrent_activation**: This is the activation function to use for the recurrent step. Common options include 'sigmoid' and 'hard_sigmoid'
* **return_sequences**: A boolean whether to return the last output in the output sequence, or the full sequence.
* **return_state**: A boolean whether to return the last state in addition to the output.
* **stateful** : A boolean, whether the last state for each sample at index i in a batch will be used as initial state for the sample of index i in the following batch.

The LSTM layer is more complex than the SimpleRNN model, making it more difficult to train and more prone to errors. Additionally, it is more sensitive to the choice of hyperparameters, such as the learning rate, number of hidden units, and the number of layers.

Thus, it is not always the best option especially for very simple Time Series.

**1. Use the Close column as the main Time Series and discard the other columns. <br>Split the msft_df time series into a train and a test set without shuffling.**

**2. Create the feature (X) set by using a sliding windows of size 20, with a step of 1 for both train and test sets.<br> Create the label (Y) set by using the next value i.e. the one after the window.**

**3. Standardize the values of the Time Series.**

*Tips: Don't forget to only compute the calculations on the Train set but apply the standardization to every set.*

**4.a. Define a sequential deep learning model architecture that contains one input layer, some LSTM layers and one output layer.<br> 4.b. Show the architecture and the parameters of the model using the `model.summary()` method.**

**5. Compile the model by adding an optimizer, a loss and, optionally, a metric using the `model.compile()` method. <br>Set the learning_rate of the optimizer to 0.005**

**6. Train the model by using the `model.fit()` method.<br>Specify a value for the number of `epochs` and the `batch_size`, and set the validation_split hyperparameter to 0.1**

**7.a. Predict the test set using the `model.predict()`.<br>7.b. Plot the predictions on a graph along with the true data.<br>7.c. Compute the RMSE.**

# Exogenous features

In this last exercise, we will try to add an Exogenous feature to the msft_df Time Series.



**1. Use the Close column as the main Time Series and the Open column as an exogenous variable. <br>Split the msft_df time series into a train and a test set without shuffling.**

Now, the feature set contains two Time Series: the Open and the Close columns of the msft dataset.

**2. Create the feature (X) set by using a sliding windows of size 20, with a step of 1 for both train and test sets.<br> Create the label (Y) set by using the next value i.e. the one after the window.**

**3. Standardize the values of the Time Series and the exogenous variable.**

*Tips: Don't forget to only compute the calculations on the Train set but apply the standardization to every set.*

The following deep learning model should contain an input layer with a shape parameter equal to (window length, number of features).<br> In our case, it should be (20,2).

**4.a. Define a sequential deep learning model architecture that contains one input layer, some LSTM layers and one output layer.<br> 4.b. Show the architecture and the parameters of the model using the `model.summary()` method.**

**5. Compile the model, train it, predict and show the results.**

# Bonus : Final Exercise

Compare four models on the lynx Time Series (you can find it [here](https://alkaline-ml.com/pmdarima/_modules/pmdarima/datasets/lynx.html)):


*   Exponential Smoothing
*   (S)ARIMA(X)
*   Machine Learning Model (LGBM)
*   Deep Learning Model (LSTM)

**1.Keep the last 25% of the Time Series for testing and use the remainder to train and validate.**<br>
**2.Try to optimize these models as much as possible.**<br>
**3.Find the best model.**